In [31]:
# Dependencies
import requests
from config_env import g_key
import time
import numpy as np
import pandas as pd
import json
import os
from IPython.core.display import display, HTML
import pymongo
from pymongo import MongoClient



# Store a search term
query = "Venezuela"

# Search for articles published between a begin and end date
begin_date = "2014-01-01"
end_date = "20190101"
offset = '&offset='

url = f"https://content.guardianapis.com/search?section=world&page-size=100&from-date={begin_date}&q={query}&api-key={g_key}&page="



In [35]:
output_list = []

for i in range(1, 20):
    query_url = url + str(i)
    try:
        articles = requests.get(query_url).json()
        if articles['response']['status'] != 'error':
            output_list.append(articles)
    except:
        pass


In [59]:
news = []
for i in range(1, 20):
    query_url = url + str(i)
    try:
        articles = requests.get(query_url).json()
        if articles['response']['status'] != 'error':
            for item in article_list['response']['results']:
                list_dict = {}
                list_dict["url"] = item["id"]
                list_dict["headline"] = item["webTitle"]
                list_dict["keywords"] = item["keywords"]
                list_dict["section"] = item["pillarName"]
                news.append(list_dict)
    except:
        pass


# print(news)

In [62]:
print(articles)

{'response': {'status': 'error', 'message': 'requested page is beyond the number of available pages'}}


In [28]:
timestr = time.strftime('%Y%m%d-%H%M%S')
output = 'out_' + timestr + '.txt'
print(f'Your data will be in {output}')

for item in output_list:
    with open(output, 'a+') as f:
        f.write(str(item))

Your data will be in out_20190220-112837.txt


In [36]:
# jfile = "out_20190220-112837.txt"

# with open(jfile, 'r') as jsonfile:
#     articles = json.load(jsonfile)
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client.venezuela

    

Mongo version 3.7.2


In [40]:
for item in output_list:
    db.news.insert_one(item)